In [1]:
import sys
sys.path.append('C:\\Users\\monty.minh\\Documents\\Optimizer\\')

In [2]:
from model import *
from model.data import Data, Results
from model.optimization import optimize
from model.preprocessing import *

Define a file path to the Excel input file
Define a file path to save the Excel output file

Data.filepath = 
Results.save_location = 


In [11]:
Data.year = 2023
Data.timeframe = [2022, 2023]

In [12]:
Data.filepath = 'C:/Users/monty.minh/Documents/Optimizer/0. Inputs/Inputs Example.xlsm'
Results.save_location = 'C:\\Users\\monty.minh\\Documents\\Optimizer\\Postprocessing Notebooks\\Results.xlsx'

In [13]:
preprocess()

In [14]:
optimize()